# Memory

In many real-world scenarios we do not have access to the ground truth state of the environment. In these cases we only get observations that have some mutual information with the true state. In this case the Markov property is violated because we are dealing with a  [POMDP](https://en.wikipedia.org/wiki/Partially_observable_Markov_decision_process) and hence information from past observations influences the estimate of the current state. Because the belief about the current state is dependent on previous observations, also the future behavior and action selection decision are dependent on them.

To respect this in our policy, it needs to be able to reason about sequences of observations. A natural way to implement this is using recurrent neural networs (RNNs) that carry an internal state that can transport information through time.

In [1]:
#define RL_TOOLS_BACKEND_ENABLE_OPENBLAS
#include <rl_tools/operations/cpu_mux.h>
#include <rl_tools/nn/optimizers/adam/instance/operations_generic.h>
#include <rl_tools/nn/operations_cpu.h>
#include <rl_tools/nn/layers/gru/operations_generic.h>
#include <rl_tools/nn/layers/sample_and_squash/operations_generic.h>
#include <rl_tools/rl/environments/memory/operations_cpu.h>
#include <rl_tools/rl/environments/pendulum/operations_cpu.h>
#include <rl_tools/nn_models/mlp/operations_generic.h>
#include <rl_tools/nn_models/random_uniform/operations_generic.h>
#include <rl_tools/nn_models/sequential/operations_generic.h>
#include <rl_tools/nn/optimizers/adam/operations_generic.h>

#include <rl_tools/rl/algorithms/sac/loop/core/config.h>
#include <rl_tools/rl/loop/steps/evaluation/config.h>
#include <rl_tools/rl/loop/steps/timing/config.h>
#include <rl_tools/rl/algorithms/sac/loop/core/operations_generic.h>
#include <rl_tools/rl/loop/steps/extrack/operations_cpu.h>
#include <rl_tools/rl/loop/steps/evaluation/operations_generic.h>
#include <rl_tools/rl/loop/steps/checkpoint/operations_cpu.h>
#include <rl_tools/rl/loop/steps/save_trajectories/operations_cpu.h>
#include <rl_tools/rl/loop/steps/timing/operations_cpu.h>

namespace rlt = rl_tools;

using DEVICE = rlt::devices::DEVICE_FACTORY<>;
using RNG = DEVICE::SPEC::RANDOM::ENGINE<>;
using T = float;
using TI = typename DEVICE::index_t;

In [2]:
#pragma cling load("openblas")

In [3]:
constexpr bool MEMORY = true;
constexpr bool MEMORY_LONG = false;

constexpr TI SEQUENCE_LENGTH = MEMORY ? (MEMORY_LONG ? 500 : 50) : 10;
constexpr TI SEQUENCE_LENGTH_PROXY = SEQUENCE_LENGTH;
constexpr TI BATCH_SIZE = MEMORY ? 4: 100;
constexpr TI NUM_CHECKPOINTS = 100;

In [4]:
struct ENVIRONMENT_PARAMETERS{
    static constexpr TI HORIZON = MEMORY_LONG ? 100 : 10;
    static constexpr T INPUT_PROBABILITY = HORIZON <= 4 ? 0.5 : (T)2/HORIZON;
    static constexpr TI EPISODE_STEP_LIMIT = 2000;
    static constexpr rlt::rl::environments::memory::Mode MODE = rlt::rl::environments::memory::Mode::COUNT_INPUT;
};


In [5]:
using MEMORY_ENVIRONMENT_SPEC = rlt::rl::environments::memory::Specification<T, TI, ENVIRONMENT_PARAMETERS>;
using MEMORY_ENVIRONMENT = rlt::rl::environments::Memory<MEMORY_ENVIRONMENT_SPEC>;
using PENDULUM_ENVIRONMENT_SPEC = rlt::rl::environments::pendulum::Specification<T, TI, rlt::rl::environments::pendulum::DefaultParameters<T>>;
using PENDULUM_ENVIRONMENT = rlt::rl::environments::Pendulum<PENDULUM_ENVIRONMENT_SPEC>;

using ENVIRONMENT = rlt::utils::typing::conditional_t<MEMORY, MEMORY_ENVIRONMENT, PENDULUM_ENVIRONMENT>;


In [6]:
struct LOOP_CORE_PARAMETERS: rlt::rl::algorithms::sac::loop::core::DefaultParameters<T, TI, ENVIRONMENT>{
    struct SAC_PARAMETERS: rlt::rl::algorithms::sac::DefaultParameters<T, TI, ENVIRONMENT::ACTION_DIM>{
        static constexpr T GAMMA = MEMORY ? 0.0 : 0.99;
        static constexpr TI ACTOR_BATCH_SIZE = BATCH_SIZE;
        static constexpr TI CRITIC_BATCH_SIZE = BATCH_SIZE;
        static constexpr TI SEQUENCE_LENGTH = SEQUENCE_LENGTH_PROXY;
        static constexpr TI CRITIC_TRAINING_INTERVAL = 1;
        static constexpr TI ACTOR_TRAINING_INTERVAL = 2;
        static constexpr bool ENTROPY_BONUS = true;
        static constexpr bool ENTROPY_BONUS_NEXT_STEP = false;

        static constexpr T TARGET_ENTROPY = MEMORY ? -4 : -1;
        static constexpr T ALPHA = 1;
        static constexpr bool ADAPTIVE_ALPHA = true;
    };
    static constexpr TI N_WARMUP_STEPS = 1000;
    static constexpr TI N_WARMUP_STEPS_CRITIC = 1000;
    static constexpr TI N_WARMUP_STEPS_ACTOR = MEMORY ? 10000: 1000;
    static constexpr TI STEP_LIMIT = 200000;
    static constexpr TI REPLAY_BUFFER_CAP = STEP_LIMIT;
    static constexpr TI ACTOR_HIDDEN_DIM = MEMORY ? (MEMORY_LONG ? 64 : 16) : 32;
    static constexpr TI ACTOR_NUM_LAYERS = 4;
    static constexpr auto ACTOR_ACTIVATION_FUNCTION = rlt::nn::activation_functions::ActivationFunction::TANH;
    static constexpr TI CRITIC_HIDDEN_DIM = ACTOR_HIDDEN_DIM;
    static constexpr TI CRITIC_NUM_LAYERS = 4;
    static constexpr auto CRITIC_ACTIVATION_FUNCTION = ACTOR_ACTIVATION_FUNCTION;
    static constexpr bool SHARED_BATCH = false;
    static constexpr TI N_ENVIRONMENTS = 1;

    struct BATCH_SAMPLING_PARAMETERS{
        static constexpr bool INCLUDE_FIRST_STEP_IN_TARGETS = true;
        static constexpr bool ALWAYS_SAMPLE_FROM_INITIAL_STATE = false;
        static constexpr bool RANDOM_SEQ_LENGTH = true;
        static constexpr bool ENABLE_NOMINAL_SEQUENCE_LENGTH_PROBABILITY = true;
        static constexpr T NOMINAL_SEQUENCE_LENGTH_PROBABILITY = 0.5;
    };

    struct ACTOR_OPTIMIZER_PARAMETERS: rlt::nn::optimizers::adam::DEFAULT_PARAMETERS_TENSORFLOW<T>{
        static constexpr T ALPHA = 1e-4;
        static constexpr bool ENABLE_BIAS_LR_FACTOR = false;
        static constexpr T BIAS_LR_FACTOR = 1;
    };
    struct CRITIC_OPTIMIZER_PARAMETERS: rlt::nn::optimizers::adam::DEFAULT_PARAMETERS_TENSORFLOW<T>{
        static constexpr T ALPHA = 1e-3;
        static constexpr bool ENABLE_BIAS_LR_FACTOR = false;
        static constexpr T BIAS_LR_FACTOR = 1;
    };
    struct ALPHA_OPTIMIZER_PARAMETERS: rlt::nn::optimizers::adam::DEFAULT_PARAMETERS_TENSORFLOW<T>{
        static constexpr T ALPHA = 1e-3;
        static constexpr bool ENABLE_BIAS_LR_FACTOR = false;
        static constexpr T BIAS_LR_FACTOR = 1;
    };
};



In [7]:
using RNG = DEVICE::SPEC::RANDOM::ENGINE<>;
using LOOP_CORE_CONFIG = rlt::rl::algorithms::sac::loop::core::Config<T, TI, RNG, ENVIRONMENT, LOOP_CORE_PARAMETERS, rlt::rl::algorithms::sac::loop::core::ConfigApproximatorsGRU>;
using LOOP_EXTRACK_CONFIG = rlt::rl::loop::steps::extrack::Config<LOOP_CORE_CONFIG>;

In [8]:
struct LOOP_EVAL_PARAMETERS: rlt::rl::loop::steps::evaluation::Parameters<T, TI, LOOP_EXTRACK_CONFIG>{
    static constexpr TI EVALUATION_INTERVAL = 1000;
    static constexpr TI NUM_EVALUATION_EPISODES = 10;
    static constexpr TI N_EVALUATIONS = LOOP_CORE_CONFIG::CORE_PARAMETERS::STEP_LIMIT / EVALUATION_INTERVAL;
};

In [9]:
using LOOP_EVAL_CONFIG = rlt::rl::loop::steps::evaluation::Config<LOOP_EXTRACK_CONFIG, LOOP_EVAL_PARAMETERS>;

In [10]:
struct LOOP_CHECKPOINT_PARAMETERS: rlt::rl::loop::steps::checkpoint::Parameters<T, TI>{
    static constexpr TI CHECKPOINT_INTERVAL_TEMP = LOOP_CORE_CONFIG::CORE_PARAMETERS::STEP_LIMIT / NUM_CHECKPOINTS;
    static constexpr TI CHECKPOINT_INTERVAL = CHECKPOINT_INTERVAL_TEMP == 0 ? 1 : CHECKPOINT_INTERVAL_TEMP;
};

In [11]:
using LOOP_CHECKPOINT_CONFIG = rlt::rl::loop::steps::checkpoint::Config<LOOP_EVAL_CONFIG, LOOP_CHECKPOINT_PARAMETERS>;

In [12]:
struct LOOP_SAVE_TRAJECTORIES_PARAMETERS: rlt::rl::loop::steps::save_trajectories::Parameters<T, TI, LOOP_CHECKPOINT_CONFIG>{
    static constexpr TI INTERVAL_TEMP = LOOP_CORE_CONFIG::CORE_PARAMETERS::STEP_LIMIT / 10;
    static constexpr TI INTERVAL = INTERVAL_TEMP == 0 ? 1 : INTERVAL_TEMP;
    static constexpr TI NUM_EPISODES = 10;
};

In [13]:
using LOOP_SAVE_TRAJECTORIES_CONFIG = rlt::rl::loop::steps::save_trajectories::Config<LOOP_CHECKPOINT_CONFIG, LOOP_SAVE_TRAJECTORIES_PARAMETERS>;
using LOOP_TIMING_CONFIG = rlt::rl::loop::steps::timing::Config<LOOP_SAVE_TRAJECTORIES_CONFIG>;
using LOOP_CONFIG = LOOP_TIMING_CONFIG;

In [ ]:
using LOOP_STATE = LOOP_CONFIG::State<LOOP_CONFIG>;
TI seed = 1;
DEVICE device;
LOOP_STATE ts;
ts.extrack_config.name = "sequential";
ts.extrack_config.population_variates = "algorithm_environment";
ts.extrack_config.population_values = "sac_memory";
rlt::malloc(device);
rlt::init(device);
rlt::malloc(device, ts);
rlt::init(device, ts, seed);
DEVICE::SPEC::RANDOM::ENGINE<> myrng;
rlt::init(device, myrng, seed);
bool done = false;
while(!done){
    if(ts.step % 1000 == 0){
        constexpr TI TEST_SEQUENCE_LENGTH = SEQUENCE_LENGTH;
        rlt::Tensor<rlt::tensor::Specification<T, TI, rlt::tensor::Shape<TI, TEST_SEQUENCE_LENGTH, 1, 2>>> test_critic_input;
        rlt::Tensor<rlt::tensor::Specification<T, TI, rlt::tensor::Shape<TI, TEST_SEQUENCE_LENGTH, 1, 1>>> test_critic_output;
        using EVALUATION_ACTOR = decltype(ts.actor_critic.actor)::CHANGE_BATCH_SIZE<TI, 1>;
        using EVALUATION_CRITIC = rlt::utils::typing::remove_reference_t<decltype(ts.actor_critic.critics[0])>::CHANGE_BATCH_SIZE<TI, 1>;
        EVALUATION_ACTOR::Buffer<1> actor_buffer;
        EVALUATION_CRITIC::Buffer<1> critic_buffer;
        rlt::malloc(device, test_critic_input);
        rlt::malloc(device, test_critic_output);
        rlt::malloc(device, actor_buffer);
        rlt::malloc(device, critic_buffer);
        auto test_actor_input = rlt::view_range(device, test_critic_input, 0, rlt::tensor::ViewSpec<2, 1>{});
        auto test_actor_output = rlt::view_range(device, test_critic_input, 1, rlt::tensor::ViewSpec<2, 1>{});
        constexpr TI N_EXAMPLES = 10;
        TI critic_correct_examples = 0;
        TI actor_correct_examples = 0;
        for(TI example_i = 0; example_i < N_EXAMPLES; example_i++){
            rlt::Mode<rlt::mode::Evaluation<>> mode;
            std::vector<TI> values;
            if(TEST_SEQUENCE_LENGTH >= 2){
                for(TI seq_i = 0; seq_i < TEST_SEQUENCE_LENGTH-1; seq_i++){
                    TI value = rlt::random::uniform_real_distribution(device.random, (T)0, (T)1, myrng) < ENVIRONMENT_PARAMETERS::INPUT_PROBABILITY ? 1 : 0;
                    values.push_back(value);
                    while(values.size() > ENVIRONMENT_PARAMETERS::HORIZON){
                        values.erase(values.begin());
                    }
                    rlt::set(device, test_critic_input, (T)value, seq_i, 0, 0);
                }
            }

//            rlt::Mode<rlt::nn::layers::gru::StepByStepMode<TI, rlt::mode::Evaluation>> mode;
//            mode.reset = true;
            while(values.size() > ENVIRONMENT_PARAMETERS::HORIZON-1){
                values.erase(values.begin());
            }
            TI pre_count = std::accumulate(values.begin(), values.end(), 0);

            for(TI input_i = 0; input_i < 2; input_i++){
//                    TI input_i = real_input_i - 1;
                rlt::set(device, test_critic_input, (T)input_i, TEST_SEQUENCE_LENGTH-1, 0, 0);
                TI count = pre_count + input_i;
                // line search
                T max_value = 0;
                bool max_value_set = false;
                TI max_action = 0;
                for(TI action_i = 0; action_i < 5; action_i++){
                    T action = ((T)action_i)/10;
                    rlt::set(device, test_critic_input, action, TEST_SEQUENCE_LENGTH-1, 0, 1);
//                    rlt::utils::assert_exit(device, rlt::get(device, test_critic_input, TEST_SEQUENCE_LENGTH-2, 0, 0) + rlt::get(device, test_critic_input, TEST_SEQUENCE_LENGTH-1, 0, 0) == count, "Count mismatch");
//                    rlt::print(device, test_critic_input);
                    rlt::evaluate(device, ts.actor_critic.actor, test_actor_input, test_actor_output, actor_buffer, myrng, mode); // to calculate the missing action
                    rlt::evaluate(device, ts.actor_critic.critics[0], test_critic_input, test_critic_output, critic_buffer, myrng, mode);
                    T value = rlt::get(device, test_critic_output, TEST_SEQUENCE_LENGTH-1, 0, 0);
                    if(!max_value_set || value > max_value){
                        max_value = value;
                        max_value_set = true;
                        max_action = action_i;
                    }
//                        std::cout << "Count " << count << " action " << action << " value: " << rlt::get(device, test_critic_output, TEST_SEQUENCE_LENGTH-1, 0, 0) << std::endl;
                }
                critic_correct_examples += max_action == count;
//                    std::cout << "Input " << input_i << " max_action " << max_action << (max_action == count ? " correct" : " incorrect") << std::endl;
                rlt::evaluate(device, ts.actor_critic.actor, test_actor_input, test_actor_output, actor_buffer, myrng, mode);
                bool actor_correct = round(rlt::get(device, test_actor_output, TEST_SEQUENCE_LENGTH-1, 0, 0) * 10) == count;
                std::cout << "Count " << count << " actor_action " << rlt::get(device, test_actor_output, TEST_SEQUENCE_LENGTH-1, 0, 0) << (actor_correct ? " ✅" : " ❌") << std::endl;
                actor_correct_examples += actor_correct;
            }
        }
        rlt::add_scalar(device, device.logger, "critic_evaluation_accuracy", critic_correct_examples / ((T)2*N_EXAMPLES));
        rlt::add_scalar(device, device.logger, "actor_evaluation_accuracy", actor_correct_examples / ((T)2*N_EXAMPLES));
        rlt::free(device, test_critic_input);
        rlt::free(device, test_critic_output);
        rlt::free(device, actor_buffer);
        rlt::free(device, critic_buffer);
    }
    done = rlt::step(device, ts);
}


Seed: 1
Extrack Experiment: "experiments/2025-10-11_06-22-15/no-hash_sequential_algorithm_environment/sac_memory/0001"


Count 5 actor_action -0.175485 ❌
Count 6 actor_action -0.196689 ❌
Count 0 actor_action -0.129703 ❌
Count 1 actor_action -0.151617 ❌
Count 0 actor_action -0.129954 ❌
Count 1 actor_action -0.151858 ❌
Count 0 actor_action -0.129913 ❌
Count 1 actor_action -0.151818 ❌
Count 0 actor_action -0.130264 ❌
Count 1 actor_action -0.152157 ❌
Count 1 actor_action -0.133836 ❌
Count 2 actor_action -0.155762 ❌
Count 0 actor_action -0.129702 ❌
Count 1 actor_action -0.151616 ❌
Count 2 actor_action -0.142455 ❌
Count 3 actor_action -0.16451 ❌
Count 2 actor_action -0.148916 ❌
Count 3 actor_action -0.170926 ❌
Count 2 actor_action -0.135248 ❌
Count 3 actor_action -0.157141 ❌
Description written to: "experiments/2025-10-11_06-22-15/no-hash_sequential_algorithm_environment/sac_memory/0001/description.txt"


Saving Trajectories to: "experiments/2025-10-11_06-22-15/no-hash_sequential_algorithm_environment/sac_memory/0001/steps/000000000000000/trajectories.json"
Checkpointing to: "experiments/2025-10-11_06-22-15/no-hash_sequential_algorithm_environment/sac_memory/0001/steps/000000000000000/checkpoint.h"


Step: 0/200000 Mean return: -6956.41 Mean episode length: 2000
Count 2 actor_action -0.137958 ❌
Count 3 actor_action -0.159942 ❌
Count 0 actor_action -0.131179 ❌
Count 1 actor_action -0.153038 ❌
Count 0 actor_action -0.12979 ❌
Count 1 actor_action -0.151701 ❌
Count 0 actor_action -0.129805 ❌
Count 1 actor_action -0.151715 ❌
Count 3 actor_action -0.14474 ❌
Count 4 actor_action -0.166892 ❌
Count 0 actor_action -0.130641 ❌
Count 1 actor_action -0.152521 ❌
Count 2 actor_action -0.137927 ❌
Count 3 actor_action -0.159912 ❌
Count 2 actor_action -0.155638 ❌
Count 3 actor_action -0.176984 ❌
Count 1 actor_action -0.138334 ❌
Count 2 actor_action -0.160392 ❌
Count 1 actor_action -0.131271 ❌
Count 2 actor_action -0.153134 ❌
Step: 1000/200000 Mean return: -6700.21 Mean episode length: 2000
Loop step: 1098, env step: 1098, SPS: 109.749 (elapsed: 10.004 s)
Loop step: 1237, env step: 1237, SPS: 13.8175 (elapsed: 20.064 s)
Loop step: 1379, env step: 1379, SPS: 14.1694 (elapsed: 30.085 s)
Loop step: 1522

Checkpointing to: "experiments/2025-10-11_06-22-15/no-hash_sequential_algorithm_environment/sac_memory/0001/steps/000000000002000/checkpoint.h"


...

Step: 50000/200000 Mean return: -677.007 Mean episode length: 2000
Loop step: 50115, env step: 50115, SPS: 13.9946 (elapsed: 3525.8 s)
Loop step: 50267, env step: 50267, SPS: 15.1388 (elapsed: 3535.84 s)
Loop step: 50366, env step: 50366, SPS: 9.89613 (elapsed: 3545.85 s)
Loop step: 50513, env step: 50513, SPS: 14.6886 (elapsed: 3555.85 s)
Loop step: 50665, env step: 50665, SPS: 15.1335 (elapsed: 3565.9 s)
Loop step: 50806, env step: 50806, SPS: 14.0394 (elapsed: 3575.94 s)
Loop step: 50929, env step: 50929, SPS: 12.2151 (elapsed: 3586.01 s)
Count 1 actor_action 0.0570827 ✅
Count 2 actor_action 0.159048 ✅
Count 3 actor_action 0.256851 ✅
Count 4 actor_action 0.362717 ✅
Count 1 actor_action 0.113259 ✅
Count 2 actor_action 0.210745 ✅
Count 4 actor_action 0.441798 ✅
Count 5 actor_action 0.53124 ✅
Count 3 actor_action 0.247643 ❌
Count 4 actor_action 0.343288 ❌
Count 2 actor_action 0.243691 ✅
Count 3 actor_action 0.346806 ✅
Count 3 actor_action 0.306628 ✅
Count 4 actor_action 0.405152 ✅
Coun

Checkpointing to: "experiments/2025-10-11_06-22-15/no-hash_sequential_algorithm_environment/sac_memory/0001/steps/000000000052000/checkpoint.h"


Step: 52000/200000 Mean return: -652.202 Mean episode length: 2000
Loop step: 52001, env step: 52001, SPS: 14.325 (elapsed: 3656.56 s)
Loop step: 52157, env step: 52157, SPS: 15.5753 (elapsed: 3666.57 s)
Loop step: 52313, env step: 52313, SPS: 15.5989 (elapsed: 3676.58 s)
Loop step: 52469, env step: 52469, SPS: 15.5935 (elapsed: 3686.58 s)
Loop step: 52625, env step: 52625, SPS: 15.5957 (elapsed: 3696.58 s)
Loop step: 52782, env step: 52782, SPS: 15.6275 (elapsed: 3706.63 s)
Loop step: 52937, env step: 52937, SPS: 15.468 (elapsed: 3716.65 s)
Count 0 actor_action -0.00552002 ✅
Count 1 actor_action 0.0986735 ✅
Count 5 actor_action 0.414248 ❌
Count 6 actor_action 0.505869 ❌
Count 3 actor_action 0.335158 ✅
Count 4 actor_action 0.434864 ✅
Count 1 actor_action 0.145825 ✅
Count 2 actor_action 0.251129 ❌
Count 3 actor_action 0.270365 ✅
Count 4 actor_action 0.3731 ✅
Count 2 actor_action 0.214115 ✅
Count 3 actor_action 0.307813 ✅
Count 1 actor_action 0.0834279 ✅
Count 2 actor_action 0.18344 ✅
Co

Checkpointing to: "experiments/2025-10-11_06-22-15/no-hash_sequential_algorithm_environment/sac_memory/0001/steps/000000000054000/checkpoint.h"


Step: 54000/200000 Mean return: -657.761 Mean episode length: 2000
Loop step: 54105, env step: 54105, SPS: 13.9822 (elapsed: 3797.1 s)
Loop step: 54250, env step: 54250, SPS: 14.4789 (elapsed: 3807.11 s)
Loop step: 54395, env step: 54395, SPS: 14.4318 (elapsed: 3817.16 s)
Loop step: 54525, env step: 54525, SPS: 12.9547 (elapsed: 3827.2 s)
Loop step: 54679, env step: 54679, SPS: 15.2915 (elapsed: 3837.27 s)
